In [49]:
import numpy as np
import pandas as pd
from IPython.display import display

In [18]:
# Load the gridset

fpv = pd.read_csv("2Dgrid.csv")

### Data Pre-processing

In [19]:
# Describing the gridset

fpv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Temp    1020 non-null   int64  
 1   Pgauge  1020 non-null   float64
 2   Fpv     1020 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 24.0 KB


In [20]:
# Pre-process
print(fpv["Temp"].max())
print(fpv["Temp"].min())

89
70


In [21]:
X_Temp = fpv.loc[:,"Temp"].drop_duplicates(keep="first")
X = X_Temp.to_numpy()
X

array([70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89], dtype=int64)

In [22]:
print(fpv["Pgauge"].max())
print(fpv["Pgauge"].min())

500.0
0.0


In [23]:
Y_Gauge = fpv.loc[:,["Pgauge"]].drop_duplicates(keep="first")
Y_array = np.array(Y_Gauge)
Y = Y_array.reshape(-1)
Y

array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
       110., 120., 130., 140., 150., 160., 170., 180., 190., 200., 210.,
       220., 230., 240., 250., 260., 270., 280., 290., 300., 310., 320.,
       330., 340., 350., 360., 370., 380., 390., 400., 410., 420., 430.,
       440., 450., 460., 470., 480., 490., 500.])

In [24]:
# to check whether Z has the require length of array
fpv["Fpv"].value_counts

<bound method IndexOpsMixin.value_counts of 0       1.0000
1       1.0007
2       1.0015
3       1.0022
4       1.0029
         ...  
1015    1.0307
1016    1.0314
1017    1.0321
1018    1.0328
1019    1.0334
Name: Fpv, Length: 1020, dtype: float64>

In [25]:
print(len(X)*len(Y))
print(len(fpv)==len(X)*len(Y))

1020
True


In [26]:
Z_fpv = fpv.loc[:,"Fpv"]
Z_array = np.array(Z_fpv)
Z = Z_array.reshape(-1)
Z

array([1.    , 1.0007, 1.0015, ..., 1.0321, 1.0328, 1.0334])

In [27]:
len(Z)

1020

### Interpolation

In [28]:
#Construct a 2-D grid and interpolate on it:

from scipy import interpolate

FPV = interpolate.interp2d(X, Y, Z, kind='linear')


In [29]:
#Testing Dataset

X1 = np.array ([76.01])
Y1 = np.array ([34.56])

In [30]:
FPVnew = FPV(X1,Y1)

In [31]:
FPVnew

array([1.0024192])

# Loading Raw Data

In [33]:
# Load Raw Data

raw = pd.read_csv("PMEOEVC.csv")

In [57]:
raw

,Date,Time,Acc. Corrected Meter Reading (Sm3),Consumption (mmBtu),Consumption (Sm3),Calorific Value (MJ/Sm3),Consumption (m3),Acc. Uncorrected Meter Reading (m3),Pressure (kPa),Temperature (°C),Conversion Factor,Contractual Conversion Factor,Z Factor
0,24/05/2022,19,NaN,0.0000,NaN,0.037852,NaN,"2,870,620.0000",0.20,17.01,0.0000,1.0,1.0075
1,24/05/2022,20,507.0000,19.1910,507.0000,0.037852,216.0,"2,870,926.0000",120.14,16.90,399.9957,1.0,0.4256
2,24/05/2022,21,"1,471.0000",36.4893,964.0000,0.037852,410.0,"2,871,336.0000",137.67,16.77,420.0009,1.0,0.4257
3,24/05/2022,22,"2,371.0000",34.0668,900.0000,0.037852,383.0,"2,871,719.0000",137.91,17.20,374.3794,1.0,0.4265
4,24/05/2022,23,"3,313.0000",35.6566,942.0000,0.037852,401.0,"2,872,120.0000",137.54,17.09,399.8637,1.0,0.4259
...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,04/07/2022,7,"842,200.0000",30.2498,803.0000,0.037671,339.0,"3,228,264.0000",138.61,15.67,333.0932,1.0,0.4238
973,04/07/2022,8,"842,989.0000",29.7224,789.0000,0.037671,333.0,"3,228,597.0000",138.62,15.79,324.1638,1.0,0.4244
974,04/07/2022,9,"843,778.0000",29.7224,789.0000,0.037671,335.0,"3,228,932.0000",137.81,16.55,332.4239,1.0,0.4263
975,04/07/2022,10,"844,590.0000",30.5889,812.0000,0.037671,346.0,"3,229,278.0000",137.56,17.05,393.7479,1.0,0.4256


In [62]:
raw.columns

Index(['Date', 'Time', 'Acc. Corrected  Meter Reading (Sm3)',
       'Consumption (mmBtu)', 'Consumption (Sm3)', 'Calorific Value (MJ/Sm3)',
       'Consumption (m3)', 'Acc. Uncorrected Meter Reading (m3)',
       'Pressure (kPa)', 'Temperature (°C)', 'Conversion Factor',
       'Contractual Conversion Factor', 'Z Factor'],
      dtype='object')

In [65]:
raw[["Consumption (m3)","Pressure (kPa)","Temperature (°C)"]]

,Consumption (m3),Pressure (kPa),Temperature (°C)
0,NaN,0.20,17.01
1,216.0,120.14,16.90
2,410.0,137.67,16.77
3,383.0,137.91,17.20
4,401.0,137.54,17.09
...,...,...,...
972,339.0,138.61,15.67
973,333.0,138.62,15.79
974,335.0,137.81,16.55
975,346.0,137.56,17.05


In [67]:
for index,row in raw.iterrows():
    

0.2 17.01
120.14 16.9
137.67 16.77
137.91 17.2
137.54 17.09
137.61 16.87
137.6 16.61
137.58 16.17
138.07 16.51
138.05 16.65
137.78 16.91
137.71 16.97
137.93 16.54
138.25 16.88
137.49 17.27
137.41 17.36
137.34 16.62
137.47 16.77
137.51 16.88
137.49 17.28
137.5 17.48
137.52 16.52
137.55 16.51
137.74 16.49
137.85 16.55
137.63 16.54
137.6 16.41
137.73 16.43
137.72 16.27
138.02 15.81
137.68 15.2
137.83 15.2
138.59 15.65
138.24 15.55
137.69 15.21
138.0 15.34
137.65 15.09
138.28 15.31
137.65 15.61
137.41 16.05
137.62 16.78
137.51 16.54
137.46 16.55
137.55 16.78
137.84 17.1
138.13 17.53
137.73 17.21
137.49 16.6
137.57 16.31
137.63 15.47
137.77 16.06
137.59 16.3
137.48 16.23
137.47 15.89
137.58 15.94
137.5 15.99
137.75 16.03
137.57 15.92
137.77 15.9
137.96 15.76
138.28 16.13
138.19 16.1
137.45 16.15
137.5 17.2
137.47 16.68
137.8 16.46
137.69 16.9
137.75 16.96
137.72 17.33
137.54 17.39
137.43 16.42
137.64 16.04
138.67 16.42
138.57 16.27
138.18 16.34
137.54 15.96
137.79 15.73
137.74 15.4
137.78 1